In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

app_path = "../Dataset/application_record.csv"   
cred_path = "../Dataset/credit_record.csv"

app = pd.read_csv(app_path)
cred = pd.read_csv(cred_path)

print("application_record shape:", app.shape)
print("credit_record shape:", cred.shape)


display(app.head())
display(cred.head())


print("\n--- application_record info ---")
display(app.info())
display(app.isna().sum().sort_values(ascending=False).head(20))

print("\n--- credit_record info ---")
display(cred.info())
display(cred.isna().sum())


def status_to_int(s):
    if s in ['C','X'] or pd.isna(s):
        return np.nan
    try:
        return int(s)
    except:
        return np.nan

cred['STATUS_NUM'] = cred['STATUS'].apply(status_to_int)


agg = cred.groupby('ID').agg(
    num_records=('STATUS', 'size'),
    num_delinq=('STATUS', lambda x: x.isin(['2','3','4','5']).sum()),
).reset_index()



grp = cred.groupby('ID')
agg = pd.DataFrame({
    'ID': grp.size().index,
    'num_records': grp.size().values,
    'num_delinq': grp.apply(lambda g: g['STATUS'].isin(['2','3','4','5']).sum()).values,
    'num_missed': grp.apply(lambda g: g['STATUS'].isin(['1','2','3','4','5']).sum()).values,
    'num_closed': grp.apply(lambda g: (g['STATUS']=='C').sum()).values,
    'num_no_loan': grp.apply(lambda g: (g['STATUS']=='X').sum()).values,
    'max_status': grp.apply(lambda g: pd.to_numeric(g['STATUS'], errors='coerce').max(skipna=True)).fillna(0).values,
    'last_status': grp.apply(lambda g: g.sort_values('MONTHS_BALANCE')['STATUS'].iloc[0]).values,
    'min_month': grp['MONTHS_BALANCE'].min().values,
    'max_month': grp['MONTHS_BALANCE'].max().values,
})


agg['fraction_delinq'] = agg['num_delinq'] / agg['num_records']


agg['risk_score'] = agg['num_delinq'] + 0.5*agg['num_missed']


agg['high_risk'] = ((agg['num_delinq'] >= 2) | (agg['max_status'] >= 3) | (agg['fraction_delinq'] > 0.2)).astype(int)


print("Target distribution (0 low-risk, 1 high-risk):")
display(agg['high_risk'].value_counts(normalize=False))
display(agg['high_risk'].value_counts(normalize=True))


merged = app.merge(agg, on='ID', how='inner')
print("Merged shape:", merged.shape)
display(merged.head())


merged.to_csv("../Dataset/merged_credit_data.csv", index=False)
print("Saved merged data to Dataset/merged_credit_data.csv")

application_record shape: (438557, 18)
credit_record shape: (1048575, 3)


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C



--- application_record info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   438557 non-null  int64  
 1   CODE_GENDER          438557 non-null  object 
 2   FLAG_OWN_CAR         438557 non-null  object 
 3   FLAG_OWN_REALTY      438557 non-null  object 
 4   CNT_CHILDREN         438557 non-null  int64  
 5   AMT_INCOME_TOTAL     438557 non-null  float64
 6   NAME_INCOME_TYPE     438557 non-null  object 
 7   NAME_EDUCATION_TYPE  438557 non-null  object 
 8   NAME_FAMILY_STATUS   438557 non-null  object 
 9   NAME_HOUSING_TYPE    438557 non-null  object 
 10  DAYS_BIRTH           438557 non-null  int64  
 11  DAYS_EMPLOYED        438557 non-null  int64  
 12  FLAG_MOBIL           438557 non-null  int64  
 13  FLAG_WORK_PHONE      438557 non-null  int64  
 14  FLAG_PHONE           438557 non-nul

None

...

high_risk
0    45570
1      415
Name: count, dtype: int64

high_risk
0    0.990975
1    0.009025
Name: proportion, dtype: float64

Merged shape: (36457, 30)


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,num_missed,num_closed,num_no_loan,max_status,last_status,min_month,max_month,fraction_delinq,risk_score,high_risk
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,1,13,1,1.0,X,-15,0,0.0,0.5,0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,1,12,1,1.0,X,-14,0,0.0,0.5,0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,...,0,7,16,0.0,X,-29,0,0.0,0.0,0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,0,0,3,0.0,X,-4,0,0.0,0.0,0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,0,0,5,0.0,X,-26,-22,0.0,0.0,0


Saved merged data to Dataset/merged_credit_data.csv


In [5]:
print(merged.shape)
print(merged["high_risk"].value_counts())
print(merged["high_risk"].value_counts(normalize=True))

print("agg size:", agg.shape)
print(agg["high_risk"].value_counts())
print(agg["high_risk"].value_counts(normalize=True))

print("\nRule components:")
print("num_delinq>=2:", (agg["num_delinq"] >= 2).mean())
print("max_status>=3:", (agg["max_status"] >= 3).mean())
print("fraction_delinq>0.2:", (agg["fraction_delinq"] > 0.2).mean())

print("any status 2-5 ever:", (agg["num_delinq"] > 0).mean())
print("any status 1-5 ever:", (agg["num_missed"] > 0).mean())
print("max_status distribution:\n", agg["max_status"].value_counts().sort_index())

(36457, 30)
high_risk
0    36074
1      383
Name: count, dtype: int64
high_risk
0    0.989494
1    0.010506
Name: proportion, dtype: float64
agg size: (45985, 13)
high_risk
0    45570
1      415
Name: count, dtype: int64
high_risk
0    0.990975
1    0.009025
Name: proportion, dtype: float64

Rule components:
num_delinq>=2: 0.00854626508644123
max_status>=3: 0.007197999347613352
fraction_delinq>0.2: 0.0033489181254756986
any status 2-5 ever: 0.014504729803196694
any status 1-5 ever: 0.11634228552788953
max_status distribution:
 max_status
0.0    40635
1.0     4683
2.0      336
3.0       88
4.0       48
5.0      195
Name: count, dtype: int64
